In [1]:
import psycopg2
import subprocess
import json
import pandas as pd
from datetime import datetime, timezone  # Added timezone
import os
import re  # For removing commas from database timestamp

# PostgreSQL Location Query

In [2]:
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "SafetyTracker"
DB_USER = "kiet"
DB_PASSWORD = "kietvo17112003"

DECRYPT_SCRIPT_PATH = "./decrypt.py"
PRIVATE_KEY = "hUotVQIdoniIfacuUNHahmnNK98GRV6+kn+sOQ=="

In [3]:
# Helper functions
def connect_db():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
        )
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL database: {e}")
        return None


def fetch_raw_movement_data(conn):
    if not conn:
        return None
    try:
        with conn.cursor() as cur:
            # Ensure you select the correct columns that you'll use
            cur.execute(
                'SELECT "LocationID", "DeviceID", "DatePublished" AS "DBDatePublished", "Payload" AS "EncryptedPayloadDB", "Description" AS "DBDescription", "StatusCode" AS "DBStatusCode" FROM "DeviceLocation" ORDER BY "LocationID";'
            )
            colnames = [desc[0] for desc in cur.description]
            rows = cur.fetchall()
            return pd.DataFrame(rows, columns=colnames)
    except psycopg2.Error as e:
        print(f"Error fetching data: {e}")
        return None


def decrypt_payload_data(payload_str, private_key_val_or_path):
    if not payload_str or pd.isna(payload_str) or payload_str.strip() == "[NULL]":
        return None

    command = [
        "python3",
        DECRYPT_SCRIPT_PATH,
        str(private_key_val_or_path),
        payload_str
    ]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        decrypted_output_str = result.stdout.strip()
        # NEW: Decrypted payload is directly the JSON object
        return json.loads(decrypted_output_str)
    except subprocess.CalledProcessError as e:
        print(f"Error during decryption script execution for payload '{payload_str[:30]}...': {e}")
        print(f"Stderr: {e.stderr}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from decrypted payload '{decrypted_output_str}': {e}")
        return None
    except FileNotFoundError:
        print(f"Error: Decryption script not found at {DECRYPT_SCRIPT_PATH}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during decryption: {e}")
        return None

In [4]:
def process_movement_data(raw_df):
    if raw_df is None:
        return pd.DataFrame()

    processed_data_list = []
    for index, row in raw_df.iterrows():
        db_location_id = row["LocationID"]
        db_device_id = row["DeviceID"]
        db_date_published_str = row["DBDatePublished"] 
        encrypted_payload_db = row["EncryptedPayloadDB"]
        db_description = row["DBDescription"]
        db_status_code = row["DBStatusCode"]

        # Initialize fields
        actual_timestamp_utc = None # From decrypted payload.timestamp
        db_date_published_utc = None # From DBDatePublished 
        latitude = None
        longitude = None
        confidence = None

        # Standardize DBDatePublished
        if db_date_published_str and isinstance(db_date_published_str, str):
            try:
                timestamp_ms_str = db_date_published_str.replace(',', '')
                timestamp_ms = float(timestamp_ms_str)
                db_date_published_utc = datetime.fromtimestamp(timestamp_ms / 1000.0, tz=timezone.utc)
            except (ValueError, TypeError) as e:
                print(f"Warning: Could not parse DBDatePublished '{db_date_published_str}' for DB LocationID {db_location_id}: {e}")

        # Decrypt Payload and Extract Coordinates
        if encrypted_payload_db and not pd.isna(encrypted_payload_db) and encrypted_payload_db.strip() != "[NULL]":
            decrypted_json = decrypt_payload_data(encrypted_payload_db, PRIVATE_KEY)

            if decrypted_json:
                latitude = decrypted_json.get("latitude")
                longitude = decrypted_json.get("longitude")
                confidence = decrypted_json.get("confidence")
                payload_timestamp_s = decrypted_json.get("timestamp") # Assumed in seconds

                if payload_timestamp_s:
                    try:
                        actual_timestamp_utc = datetime.fromtimestamp(payload_timestamp_s, tz=timezone.utc)
                    except (ValueError, TypeError) as e:
                        print(f"Warning: Could not parse decrypted payload.timestamp '{payload_timestamp_s}' for DB LocationID {db_location_id}: {e}")
                else:
                    print(f"Warning: 'timestamp' not found in decrypted payload for DB LocationID {db_location_id}")

                if latitude is None or longitude is None:
                    print(f"Warning: 'latitude' or 'longitude' not found in decrypted payload for DB LocationID {db_location_id}")
            else:
                print(f"Warning: Failed to decrypt payload for DB LocationID {db_location_id}")
        elif encrypted_payload_db and encrypted_payload_db.strip() == "[NULL]":
            print(f"Info: EncryptedPayloadDB is '[NULL]' string for DB LocationID {db_location_id}. No decryption attempted.")

        processed_data_list.append({
            "LocationID": db_location_id, # From DB
            "DeviceID": db_device_id,     # From DB
            "TimestampUTC": actual_timestamp_utc, # This is the primary timestamp from decrypted payload
            "Latitude": latitude,
            "Longitude": longitude,
            "Confidence": confidence,
            "Description": db_description, # From DB
            "StatusCode": db_status_code,   # From DB
            "DBDatePublishedUTC": db_date_published_utc, # Timestamp from DB, converted
            "EncryptedPayloadDB": encrypted_payload_db
        })

    return pd.DataFrame(processed_data_list)

## Main Execution

In [5]:
if not os.path.exists(DECRYPT_SCRIPT_PATH):
    print(f"CRITICAL ERROR: Decryption script not found at {DECRYPT_SCRIPT_PATH}")
    exit()

db_connection = connect_db()
if db_connection:
    raw_df_from_db = fetch_raw_movement_data(db_connection)
    db_connection.close()

    if raw_df_from_db is not None and not raw_df_from_db.empty:
        print(f"Fetched {len(raw_df_from_db)} raw movement records from database.")
        
        processed_df = process_movement_data(raw_df_from_db.copy())
        print("\n--- Processed Movement Data (First 5 Records) ---")
        print(processed_df.head())
        print("\n--- Processed Movement Data (Last 5 Records) ---")
        print(processed_df.tail())
        print("\n--- Processed Movement Data Info ---")
        processed_df.info()

        # Cleaned_df uses TimestampUTC (from decrypted payload) as the critical timestamp
        cleaned_df = processed_df.dropna(subset=['TimestampUTC', 'Latitude', 'Longitude'])
        print(f"\nRemoved {len(processed_df) - len(cleaned_df)} rows with missing essential data (TimestampUTC, Latitude, Longitude) after processing.")
        print(f"Final dataset size for further processing: {len(cleaned_df)} records.")

        if not cleaned_df.empty:
            output_filename_parquet = "danang_movement_processed_decrypted.parquet"
            try:
                cleaned_df.to_parquet(output_filename_parquet, index=False)
                print(f"\nProcessed and decrypted movement data saved to '{output_filename_parquet}'.")
            except Exception as e:
                print(f"\nError saving processed data to Parquet: {e}")

            output_filename_csv = "danang_movement_processed_decrypted.csv"
            try:
                cleaned_df.to_csv(output_filename_csv, index=False)
                print(f"Processed and decrypted movement data saved to '{output_filename_csv}'.")
            except Exception as e:
                print(f"\nError saving processed data to CSV: {e}")
        else:
            print("\nNo data left after cleaning. Output files not saved.")
    else:
        print("No raw movement data fetched from database or DataFrame is empty.")
else:
    print("Could not connect to the database. Exiting.")

print("\nProcessing finished.")

Fetched 6861 raw movement records from database.
Error during decryption script execution for payload 'LRCGhwMEW+A3mSXNUcbYP/3Ev1Jqr/...': Command '['python3', './decrypt.py', 'hUotVQIdoniIfacuUNHahmnNK98GRV6+kn+sOQ==', 'LRCGhwMEW+A3mSXNUcbYP/3Ev1Jqr/XChRUYsx1c7BIQyRS13JjD0O6Txk56NlOZ5SHhCFGD/YEbbDb8s91eSkg7fsG8nJiA7zGzi9OJ0KWF9XWHZUqwfw==']' returned non-zero exit status 1.
Stderr: Traceback (most recent call last):
  File "/home/kiet/projects/learning/Python/FinalProject/Dataset/HistoricalMovement/./decrypt.py", line 98, in <module>
    decrypt = tag.decrypt_message(sys.argv[2])
  File "/home/kiet/projects/learning/Python/FinalProject/Dataset/HistoricalMovement/./decrypt.py", line 88, in decrypt_message
    decrypted = self.__decrypt_payload(enc_data, derived_key, tag)
  File "/home/kiet/projects/learning/Python/FinalProject/Dataset/HistoricalMovement/./decrypt.py", line 61, in __decrypt_payload
    return decryptor.update(enc_data) + decryptor.finalize()
                            